# Jupyter Notebook da Integradora 4
## Aluno: Thiago Medina
## Orientadora: Isabel H. Manssour

Abaixo, os imports necessário para a realização de todo o notebook

In [157]:
#1
import sys
!{sys.executable} -m pip install nltk
!{sys.executable} -m pip install unidecode
!{sys.executable} -m pip install tweepy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install openpyxl
!{sys.executable} -m pip install xlrd
!{sys.executable} -m pip install openpyxl
!{sys.executable} -m pip install ssl

import ssl
import tweepy as tw
import pandas as pd
import matplotlib
import nltk
from nltk.tokenize import TweetTokenizer
import string
import re
from unidecode import unidecode

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import ComplementNB
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score


from sklearn import svm
from sklearn.naive_bayes import GaussianNB

import numpy as np

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You sh

    ERROR: Command errored out with exit status 1:
     command: /Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/wx/c047tw591yz4cytxbj_26lv40000gp/T/pip-install-bdug9sug/ssl/setup.py'"'"'; __file__='"'"'/private/var/folders/wx/c047tw591yz4cytxbj_26lv40000gp/T/pip-install-bdug9sug/ssl/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/wx/c047tw591yz4cytxbj_26lv40000gp/T/pip-pip-egg-info-csvejgj8
         cwd: /private/var/folders/wx/c047tw591yz4cytxbj_26lv40000gp/T/pip-install-bdug9sug/ssl/
    Complete output (6 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/private/var/folders/wx/c047tw591yz4cytxbj_26lv40000gp/T/pip-install-bdug9sug/ssl/setup.py", line 33
        print 'looki

## O trabalho todo está separado em 3 partes:
### 1- Coleta dos tweets
### 2- Pré-processamento dos tweets.
### 3- Treinamento e teste por parte do algoritmo

Iniciamos com a coleta dos tweets:

Necessário possuir suas chaves próprias para a coleta de tweets.

In [158]:
#2
consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""

In [159]:
#3
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tw.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

public_tweets = api.home_timeline()

### Query para a coleta de tweets.

Altere a query de pesquisa (pode-se utilizar operadores lógicos). Altere também a quantidade de tweets que deseja coletar.

In [160]:
#4
#query = '(#masterchef)  OR(#MasterChefBR) OR (#masterchefbrasil)  -filter:retweets'
#query = '(#bbb21) -filter:retweets'
query = '(#covid19) OR (#covid) OR (#coronavirus) -filter:retweets'
#query = '(#educacao AND (-@mpsnet AND -@mpsnet2)) -filter:retweets'

cursor_tweets = tw.Cursor(api.search,
            q=query,lang = 'pt',tweet_mode='extended').items(1)

Utilizamos apenas a coluna com o texto dos tweets e a data de criação dos mesmos.

In [161]:
#5
tweets_dict = {}
tweets_dict = tweets_dict.fromkeys(['created_at', 'full_text'])

In [162]:
#6
for tweet in cursor_tweets:
    for key in tweets_dict.keys():
        try:
            twvalue = tweet._json[key]
            tweets_dict[key].append(twvalue)
        except KeyError:
            twvalue = ""
            if(tweets_dict[key] is None):
                tweets_dict[key] = [twvalue]
            else:
                tweets_dict[key].append(twvalue)
        except:
            tweets_dict[key] = [twvalue]
        #print("tweets_dict[key]: {} - tweet[key]: {}".format(tweets_dict[key],  twvalue))

Abaixo, listamos os tweets coletados.

In [163]:
#7
tweets_dict

{'created_at': ['Sun Aug 01 17:49:32 +0000 2021'],
 'full_text': ['Sociedade\nHá mais 28 mortes e 1.513 infecções por Coronavírus no país\n#covid19 #mocambique \nhttps://t.co/sZJ04pgv3n']}

Adicionamos esses tweets a um DataFrame, para poder manipular os dados.

In [164]:
#8
dfTweets = pd.DataFrame.from_dict(tweets_dict)

Mostramos os primeiros 5 dados do nosso DataFrame:


In [165]:
#9
dfTweets.head()

,created_at,full_text
0,Sun Aug 01 17:49:32 +0000 2021,Sociedade\nHá mais 28 mortes e 1.513 infecções...


E aqui, alteramos os nomes das colunas para português.

In [166]:
#10
dfTweets.rename(columns={"full_text": "Texto", "created_at": "Data de criação"})

,Data de criação,Texto
0,Sun Aug 01 17:49:32 +0000 2021,Sociedade\nHá mais 28 mortes e 1.513 infecções...


Por fim, adicionamos uma coluna chamada "Sentimento", aonde será colocada a anotação manual do sentimento de cada um desses tweets, e salvamos o DataFrame em um arquivo Excel, aonde é mais simples a manipulação.

In [167]:
#11
dfTweets.insert(2, "Sentimento1", " ")
dfTweets.insert(3, "Sentimento2", " ")
dfTweets.insert(4, "SentimentoFinal", " ")
dfTweets.to_excel("/Users/gustavoduarte/Desktop/TCC-analise-sentimentos-main/TweetsColetadosCovid.xlsx")

## A partir daqui, iremos dividir e duplicar os datasets binários!!

In [180]:
#ler o arquivo
tweetsToBinary = pd.read_excel('/Users/gustavoduarte/Desktop/TCC-analise-sentimentos-main/TabelaNetflix.xlsx', index_col=0,  engine='openpyxl') 

In [181]:
newTweets = tweetsToBinary[tweetsToBinary['SentimentoFinal'] != 0]
#newTweets.to_excel("/Users/gustavoduarte/Desktop/TCC-analise-sentimentos-main/TabelaNetflixBin.xlsx")


## A partir daqui, iremos balancear o dataset já existente entre neutros, positivos e negativos!!

In [286]:
#ler o arquivo
tweetsToBalance = pd.read_excel('/Users/gustavoduarte/Desktop/TCC-analise-sentimentos-main/TabelaCovidTeste.xlsx', index_col=0,  engine='openpyxl')
tweetsNeutral = tweetsToBalance[tweetsToBalance['SentimentoFinal'] == 0]
tweetsPositive = tweetsToBalance[tweetsToBalance['SentimentoFinal'] == 1]
tweetsNegative = tweetsToBalance[tweetsToBalance['SentimentoFinal'] == 2]

print(len(tweetsNeutral))
print(len(tweetsPositive))
print(len(tweetsNegative))
#df.index[[1,3]], inplace=True
newTweetNeutral = tweetsNeutral.drop(tweetsNeutral.index[[range(0,41)]])
newTweetPositive = tweetsPositive
newTweetNegative = tweetsNegative.drop(tweetsNegative.index[[range(0,82)]])
print(len(newTweetNeutral))
print(len(newTweetPositive))
print(len(newTweetNegative))

newTweetList = pd.concat([newTweetNeutral, newTweetPositive, newTweetNegative])
print(len(newTweetList))
# shuffle the DataFrame rows
df = newTweetList.sample(frac = 1)
df
#df.to_excel("/Users/gustavoduarte/Desktop/TCC-analise-sentimentos-main/TabelaCovidTesteBalenceada.xlsx")


100
59
141
59
59
59
177


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexes/base.py:4107: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


## A partir daqui, iremos balancear o dataset binário!!

In [287]:
#ler o arquivo
tweetsToBinaryBalance = pd.read_excel('/Users/gustavoduarte/Desktop/TCC-analise-sentimentos-main/TabelaCovidTesteBalenceada.xlsx', index_col=0,  engine='openpyxl')

In [288]:
newTweets = tweetsToBinaryBalance[tweetsToBinaryBalance['SentimentoFinal'] != 0]
#newTweets.to_excel("/Users/gustavoduarte/Desktop/TCC-analise-sentimentos-main/TabelaCovidTesteBinBalanceado.xlsx")

## A partir daqui, já deve ter sido realizada a anotação manual dos tweets!!

### Tendo feita a anotação manual dos tweets, que no caso desse trabalho foi feito por 3 pessoas, adicionamos a tabela excel em uma variavel chamada tweets.

In [509]:
#12
tweets= pd.read_excel('/Users/gustavoduarte/Desktop/TCC-analise-sentimentos-main/TabelaCovidTesteBinBalanceado.xlsx', index_col=0,  engine='openpyxl') 

Abaixo, vemos como ficou nossa tabela.
Lembrando que tweets classificados como 0 são negativos, como 1 são positivos e como 2 são neutros.

In [510]:
#13
tweets = tweets.reset_index()
tweets

,index,full_text,Sentimento1,Sentimento2,SentimentoFinal
0,195,🤔 Não sabia desses dados? Curtiu a informação?...,2,2,2
1,294,Até agora temos no Brasil:\n\n-14237078 casos ...,2,2,2
2,211,COMO #SINCRONIZAR OS #CONTATOS DO #GOOGLE NO i...,2,2,2
3,140,"Comando Conjunto Leste (C Cj L), em parceria c...",1,1,1
4,194,A seção voltada aos povos indígenas do guia Im...,2,2,2
...,...,...,...,...,...
113,206,SAIU! JAILBREAK CHECKRA1N PARA IOS 14.4.2 | FA...,2,2,2
114,151,💉Vacinação em Macaé &lt;&lt;\n\nFique ligado n...,1,2,1
115,200,INSTALOU O iOS 14? SAIBA COMO VOLTAR PARA O iO...,2,2,2
116,69,"Em Cuba, a vacina é SOBERANA. Até o nome é lin...",1,1,1


## A seguir, iniciaremos o trabalho de pré-processamento dos tweets dos nossos datasets.

### O pré-processamento tem como objetivo deixar o texto presente nos tweets mais "limpo". Está sendo feito com apenas 1 dataset, apenas como exemplo, mas a técnica foi aplicada nos 3 datasets. 

Vamos utilizar um tweet como exemplo para que possa ser verificada o resultado de cada uma das tarefas de pré-processamento.

In [511]:
#14
exemploPreProcess = (tweets.at[2,'full_text'])
print (exemploPreProcess)

COMO #SINCRONIZAR OS #CONTATOS DO #GOOGLE NO iPhone em 2021 | INFALÍVEL
https://t.co/0c4D9ZKajo

 #google #CoronaVac #coronavirus #apple #ios #iphone #AppleRetweetBot #vemvacina #bbb21


Para a tokenização dos tweets (separar a frase em pequenos termos que façam sentido), utilizamos uma biblioteca específica para tweets. Ela auxilia na separação de emojis, hashtags, menções e outras questões próprias do Twitter.

In [512]:
#15
tweet_tokenize = TweetTokenizer()
tweet_tokenize.tokenize(exemploPreProcess)

['COMO',
 '#SINCRONIZAR',
 'OS',
 '#CONTATOS',
 'DO',
 '#GOOGLE',
 'NO',
 'iPhone',
 'em',
 '2021',
 '|',
 'INFALÍVEL',
 'https://t.co/0c4D9ZKajo',
 '#google',
 '#CoronaVac',
 '#coronavirus',
 '#apple',
 '#ios',
 '#iphone',
 '#AppleRetweetBot',
 '#vemvacina',
 '#bbb21']

A seguir, baixamos uma base de "stopwords" em português. Stopwords são palavras que não agregam sentido ao texto. Artigos e preposições são exemplos de stopwords.

In [513]:
#16
try:
     _create_unverified_https_context =     ssl._create_unverified_context
except AttributeError:
     pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
    
nltk.download('stopwords')
def RemoveStopWords(tweet):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    textInWords = []
    for word in tweet.lower().split():
        if (word not in stopwords) and (word.isdigit() == False): 
            textInWords.append(word)

    res = ' '.join(map(str, textInWords))
    return res

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gustavoduarte/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Podemos ver abaixo como fica o tweet após a remoção de suas stopwords.

In [514]:
#17
RemoveStopWords(exemploPreProcess)

'#sincronizar #contatos #google iphone | infalível https://t.co/0c4d9zkajo #google #coronavac #coronavirus #apple #ios #iphone #appleretweetbot #vemvacina #bbb21'

Para fazer a stemização de cada tweet (deixar apenas a raiz(radical) de cada termo), utilizamos uma biblioteca própria que nos ajuda a fazer isso. Essa bibloteca é específica para português. 

In [515]:
#18
nltk.download('rslp')

[nltk_data] Downloading package rslp to
[nltk_data]     /Users/gustavoduarte/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [516]:
#19
#stemmer = nltk.stem.porter.PorterStemmer()
stemmer = nltk.stem.RSLPStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tweet):
    textInWords = []
    for word in tweet.lower().split():
        textInWords.append(stemmer.stem(word))
    res = ' '.join(map(str, textInWords))
    return res


Podemos ver abaixo como fica o tweet após a sua stemização.

In [517]:
#20
stem_tokens(exemploPreProcess)

'com #sincron os #contat do #googl no iphon em 2021 | infal https://t.co/0c4d9zkaj #googl #coronavac #coronaviru #appl #io #iphon #appleretweetbot #vemvacin #bbb21'

Abaixo, temos uma função de "limpeza". Nela, são excluídas quaisquer caracteres que não são letras. Também são excluídos links, hashtags e emojis. Todas as letras são passadas para minúsculo. Além disso, são excluídos dígitos.

In [518]:
#21
def Limpa_dados(tweet): ##M MUDAR NOME DESSA FUNCAO
    # remove links, pontos, virgulas,ponto e virgulas dos tweets E ACENTOS
    tweet = unidecode(re.sub(r"http\S+", "", tweet).lower().replace('.','')
                      .replace(';','').replace('+','').replace(']','').replace('[','').replace('\'','').replace(';','').replace('%','').replace('-',' ').replace(',',' ')
                      .replace('\n','').replace('\"','').replace(')','').replace('(','').replace('$','').replace('?','')
                      .replace('_','').replace(':','').replace('!','').replace('|','').replace('/','').replace(')',''))
    tweet = unidecode(re.sub(r"@[A-Za-z0-9]+", "", tweet))
    tweet = unidecode(re.sub(r"\#[A-Za-z0-9]+", "", tweet))
    
    tweet = unidecode(re.sub(r"[0-9]+[A-Za-z0-9]+", "", tweet)) #Remove digitos
    tweet = tweet.replace('  ', ' ') #Corrige espaços feitos a mais.
    tweet = tweet.replace('[?]', ' ') #Corrige outros errinhos.
    return (tweet)

Abaixo, vemos o tweets após a sua "limpeza".

In [519]:
#22
Limpa_dados(exemploPreProcess)

'como os do no iphone em  infalivel     '

Por fim, executamos as 3 funções de pré-processamento e adicionamos a uma nova tabela Excel.

In [520]:
#23
#PreProcessa todos os tweets:

qtdTweets = len(tweets)
for tweet in range (qtdTweets):
    tweets.at[tweet, 'full_text'] =  Limpa_dados(str(tweets.at[tweet, 'full_text']))
    tweets.at[tweet, 'full_text'] =  RemoveStopWords(str(tweets.at[tweet, 'full_text']))
    tweets.at[tweet, 'full_text'] =  stem_tokens(str(tweets.at[tweet, 'full_text']))

In [521]:
#24
#tweets.to_excel("/Users/gustavoduarte/Desktop/TCC-analise-sentimentos-main/TabelaCovidTesteBinBalanceadoPreProcessados.xlsx")

## Começaremos então o processo de treinamento por parte dos algoritmos.

Primeiramente, pegamos os dados de treinamento e teste. Aqui estaremos treinando o algoritmo com os dados de covid (700 tweets) e testando com os dados de covid (300 tweets). Os dados vem de tabelas diferentes e são diferentes.

In [522]:
dadosTreino = pd.read_excel('/Users/gustavoduarte/Desktop/TCC-analise-sentimentos-main/TabelaCovidTreinoPreProcessados.xlsx', engine='openpyxl').fillna(' ')
dadosTeste = pd.read_excel('/Users/gustavoduarte/Desktop/TCC-analise-sentimentos-main/TabelaCovidTestePreProcessados.xlsx', engine='openpyxl').fillna(' ')

In [523]:
dadosTreino.head()

,Unnamed: 0,full_text,Sentimento1,Sentimento2,SentimentoFinal
0,0,notic hor vai vacin nest feri enta fiqu atent ...,2,2,2
1,1,import pal expliqu importanc med prevenca cont...,2,2,2
2,2,veicul cont sistem mens som informaco utel pop...,2,2,2
3,3,beij temp cris sar arapiun edn arapiun devid v...,2,2,2
4,4,despach intim presid jair bolsonar ministr sau...,2,2,2


In [524]:
dadosTeste.head()

,Unnamed: 0,full_text,Sentimento1,Sentimento2,SentimentoFinal
0,0,assum risc tom morr pesquis demonstr cloroquin...,0,0,0
1,1,ind imag hospit gtb nov delh vari paci esta es...,0,2,0
2,2,brasil numer suspeit cas confirm numer mort re...,2,2,2
3,3,boletim check up saudeol vej aco melhor saud d...,2,2,2
4,4,catorz est distrit feder mant ocupaca util acim,0,0,0


Vamos pegar os textos dos tweets de treinamento e teste e seus respectivos sentimentos.

In [525]:
tweetsParaTreino = dadosTreino['full_text'].values
classesParaTreino = dadosTreino['SentimentoFinal'].values
tweetsParaTeste = dadosTeste['full_text'].values
classesParaTeste = dadosTeste['SentimentoFinal'].values


Utilizamos a técnica Holdout, separando os dados em 70% para treinamento e 30% para teste.

In [526]:
qtdTweetsTreino = len(dadosTreino)
qtdTweetsTreino

700

In [527]:
qtdTweetsTeste = len(dadosTeste)
qtdTweetsTeste

300

Aqui, instanciamos o vetorizador dos tweets. Ele é responsável pela transformação dos dados textuais em um formato que o algoritmo entenda. É utilizada a técnica Bag Of Words, aonde as palavras viram colunas em uma tabela e cada tweet terá uma representação de 0's e 1's. 

In [528]:
# Instancia o objeto que faz a vetorização dos dados de texto:
tweet_tokenizer = TweetTokenizer() 
vectorizer = CountVectorizer(analyzer="word", tokenizer=tweet_tokenizer.tokenize)

Abaixo, vetorizamos os dados de treinamento

In [529]:
vect_tweetsTreino = vectorizer.fit_transform(tweetsParaTreino) 

Finalmente, faremos o treinamento do algoritmo. Utilizamos o algoritmo Multinomial Naive Bayes, o SVM.

In [530]:
classificador = MultinomialNB()
classificadorBerno = BernoulliNB()
classificadorComplement = ComplementNB()
classificador.fit(vect_tweetsTreino, classesParaTreino)  
classificadorBerno.fit(vect_tweetsTreino, classesParaTreino)  
classificadorComplement.fit(vect_tweetsTreino, classesParaTreino)  

classificadorSVM = svm.SVC(kernel='linear')
classificadorSVM.fit(vect_tweetsTreino, classesParaTreino)

classificadorLR = LogisticRegression(random_state=0).fit(vect_tweetsTreino, classesParaTreino)


Já tendo o algoritmo sido treinado, faremos a vetorização dos dados de teste.

In [531]:
vect_tweetsTeste = vectorizer.transform(tweetsParaTeste) 

Abaixo, fazemos a predição dos dados de teste por parte do algoritmo já treinado.

In [532]:
result = classificador.predict(vect_tweetsTeste)
resultBerno = classificadorBerno.predict(vect_tweetsTeste)
resultComplement = classificadorComplement.predict(vect_tweetsTeste)
resultSVM = classificadorSVM.predict(vect_tweetsTeste)
resultLR = classificadorLR.predict(vect_tweetsTeste)

### Agora já possuímos o algoritmo treinado e os dados para verificar o resultado nos testes. Vamos prosseguir vendo o resultado das métricas.

Vamos ver a acurácia primeiramente.

In [533]:
acc = accuracy_score(classesParaTeste, result) * 100
print("{}{} ".format(acc.round(2), "%"))

acc2 = accuracy_score(classesParaTeste, resultSVM) * 100
print("{}{} ".format(acc2.round(2), "%"))

acc3 = accuracy_score(classesParaTeste, resultLR) * 100
print("{}{} ".format(acc3.round(2), "%"))

acc4 = accuracy_score(classesParaTeste, resultBerno) * 100
print("{}{} ".format(acc3.round(2), "%"))

acc5 = accuracy_score(classesParaTeste, resultComplement) * 100
print("{}{} ".format(acc3.round(2), "%"))

63.0% 
60.33% 
63.67% 
63.67% 
63.67% 


Vamos verificar a matriz de confusão agora. Ela tem o objetivo de ver em que classes o algoritmo conseguiu classificar melhor.

In [534]:
print (pd.crosstab(classesParaTeste, result, rownames=['Real'], colnames=['Predito'], margins=True))

#print (pd.crosstab(classesParaTeste, resultSVM, rownames=['Real'], colnames=['Predito'], margins=True))

#print (pd.crosstab(classesParaTeste, resultLR, rownames=['Real'], colnames=['Predito'], margins=True))

print (pd.crosstab(classesParaTeste, resultBerno, rownames=['Real'], colnames=['Predito'], margins=True))

print (pd.crosstab(classesParaTeste, resultComplement, rownames=['Real'], colnames=['Predito'], margins=True))


Predito    0   1    2  All
Real                      
0         83   6   11  100
1         20  22   17   59
2         41  16   84  141
All      144  44  112  300
Predito   0  1    2  All
Real                    
0        42  2   56  100
1        13  2   44   59
2        10  1  130  141
All      65  5  230  300
Predito    0   1    2  All
Real                      
0         76   8   16  100
1         16  31   12   59
2         42  19   80  141
All      134  58  108  300


Vamos verificar, por fim, as outras métricas.

In [535]:
print(classification_report(classesParaTeste, result, labels = [0,1,2]))

#print(classification_report(classesParaTeste, resultSVM, labels = [0,1,2]))

#print(classification_report(classesParaTeste, resultLR, labels = [0,1,2]))

print(classification_report(classesParaTeste, resultBerno, labels = [0,1,2]))

print(classification_report(classesParaTeste, resultComplement, labels = [0,1,2]))

              precision    recall  f1-score   support

           0       0.58      0.83      0.68       100
           1       0.50      0.37      0.43        59
           2       0.75      0.60      0.66       141

    accuracy                           0.63       300
   macro avg       0.61      0.60      0.59       300
weighted avg       0.64      0.63      0.62       300

              precision    recall  f1-score   support

           0       0.65      0.42      0.51       100
           1       0.40      0.03      0.06        59
           2       0.57      0.92      0.70       141

    accuracy                           0.58       300
   macro avg       0.54      0.46      0.42       300
weighted avg       0.56      0.58      0.51       300

              precision    recall  f1-score   support

           0       0.57      0.76      0.65       100
           1       0.53      0.53      0.53        59
           2       0.74      0.57      0.64       141

    accuracy        

## Agora, faremos o treinamento com os dados da tabela de assuntos diversos (700 tweets). A tabela de teste segue sendo a mesma.

Esse processo é semelhante ao feito anteriormente, então não há necessidade de explicar cada etapa.

In [536]:
dadosTreinoDiversos = pd.read_excel('/Users/gustavoduarte/Desktop/TCC-analise-sentimentos-main/TabelaDiversosPreProcessados.xlsx', engine='openpyxl').fillna(' ')

In [537]:
dadosTreinoDiversos.head()

,Unnamed: 0,full_text,Sentimento1,Sentimento2,SentimentoFinal
0,0,ant malest coa falt dialog med xunt propux abr...,0,0,0
1,1,pra gast pod gast pouc acess,2,2,2
2,2,oposica reform tribut desconhec lobby,0,2,2
3,3,importanc poupanc composica setor,1,1,1
4,4,"""pib governo"" err",0,2,0


In [538]:
tweetsParaTreinoDiversos = dadosTreinoDiversos['full_text'].values
classesParaTreinoDiversos = dadosTreinoDiversos['SentimentoFinal'].values

In [539]:
qtdTweetsTreino = len(dadosTreinoDiversos)
qtdTweetsTreino

700

In [540]:
vect_tweetsTreinoDiversos = vectorizer.fit_transform(tweetsParaTreinoDiversos)  #treina com diversos

In [541]:
classificadorDiversos = MultinomialNB()
classificadorDiversos.fit(vect_tweetsTreinoDiversos, classesParaTreinoDiversos) 

classificadorDiversosBerno = BernoulliNB()
classificadorDiversosBerno.fit(vect_tweetsTreinoDiversos, classesParaTreinoDiversos) 

classificadorDiversosComplement = ComplementNB()
classificadorDiversosComplement.fit(vect_tweetsTreinoDiversos, classesParaTreinoDiversos) 

classificadorSVMDiv = svm.SVC(kernel='linear')
classificadorSVMDiv.fit(vect_tweetsTreinoDiversos, classesParaTreinoDiversos)

classificadorLRDiv = LogisticRegression(random_state=0).fit(vect_tweetsTreinoDiversos, classesParaTreinoDiversos)

In [542]:
vect_tweetsTeste = vectorizer.transform(tweetsParaTeste) 

In [543]:
resultDiversos = classificadorDiversos.predict(vect_tweetsTeste)

resultDiversosBerno = classificadorDiversosBerno.predict(vect_tweetsTeste)

resultDiversosComplement = classificadorDiversosComplement.predict(vect_tweetsTeste)

resultDiversosSVM = classificadorSVMDiv.predict(vect_tweetsTeste)

resultDiversosLR = classificadorLRDiv.predict(vect_tweetsTeste)

In [544]:
acc = accuracy_score(classesParaTeste, resultDiversos) * 100
print("{}{} ".format(acc.round(2), "%"))

acc2 = accuracy_score(classesParaTeste, resultDiversosSVM) * 100
print("{}{} ".format(acc2.round(2), "%"))

acc3 = accuracy_score(classesParaTeste, resultDiversosLR) * 100
print("{}{} ".format(acc3.round(2), "%"))

acc4 = accuracy_score(classesParaTeste, resultDiversosBerno) * 100
print("{}{} ".format(acc.round(2), "%"))

acc5 = accuracy_score(classesParaTeste, resultDiversosComplement) * 100
print("{}{} ".format(acc.round(2), "%"))

55.0% 
61.67% 
58.0% 
55.0% 
55.0% 


In [546]:
print (pd.crosstab(classesParaTeste, resultDiversos, rownames=['Real'], colnames=['Predito'], margins=True))

#print (pd.crosstab(classesParaTeste, resultDiversosSVM, rownames=['Real'], colnames=['Predito'], margins=True))

#print (pd.crosstab(classesParaTeste, resultDiversosLR, rownames=['Real'], colnames=['Predito'], margins=True))

print (pd.crosstab(classesParaTeste, resultDiversosBerno, rownames=['Real'], colnames=['Predito'], margins=True))

print (pd.crosstab(classesParaTeste, resultDiversosComplement, rownames=['Real'], colnames=['Predito'], margins=True))

Predito   0   1    2  All
Real                     
0        40  17   43  100
1         5  32   22   59
2        19  29   93  141
All      64  78  158  300
Predito  0   1    2  All
Real                    
0        5   3   92  100
1        0   6   53   59
2        0   3  138  141
All      5  12  283  300
Predito   0    1    2  All
Real                      
0        52   25   23  100
1         6   39   14   59
2        31   40   70  141
All      89  104  107  300


In [547]:
print(classification_report(classesParaTeste, resultDiversos, labels = [0,1,2]))
print(classification_report(classesParaTeste, resultDiversosBerno, labels = [0,1,2]))
print(classification_report(classesParaTeste, resultDiversosComplement, labels = [0,1,2]))


#print(classification_report(classesParaTeste, resultDiversosSVM, labels = [0,1,2]))

#print(classification_report(classesParaTeste, resultDiversosLR, labels = [0,1,2]))

              precision    recall  f1-score   support

           0       0.62      0.40      0.49       100
           1       0.41      0.54      0.47        59
           2       0.59      0.66      0.62       141

    accuracy                           0.55       300
   macro avg       0.54      0.53      0.53       300
weighted avg       0.57      0.55      0.55       300

              precision    recall  f1-score   support

           0       1.00      0.05      0.10       100
           1       0.50      0.10      0.17        59
           2       0.49      0.98      0.65       141

    accuracy                           0.50       300
   macro avg       0.66      0.38      0.31       300
weighted avg       0.66      0.50      0.37       300

              precision    recall  f1-score   support

           0       0.58      0.52      0.55       100
           1       0.38      0.66      0.48        59
           2       0.65      0.50      0.56       141

    accuracy        

## Agora faremos a classificação em tabelas binárias, aonde há apenas as classes positiva e negativa (1 e 0).

Para tal, foram feitas cópias dos 3 datasets e excluídos os tweets neutros nessas cópias. Assim, faremos o treinamento e teste novamente nestes datasets.
Este processo já foi explicado anteriormente.

In [563]:
dadosTreinoBin = pd.read_excel('/Users/gustavoduarte/Desktop/TCC-analise-sentimentos-main/TabelaCovidTreinoBinPreProcessados.xlsx', engine='openpyxl').fillna(' ')
dadosTesteBin = pd.read_excel('/Users/gustavoduarte/Desktop/TCC-analise-sentimentos-main/TabelaCovidTesteBinPreProcessados.xlsx',engine='openpyxl').fillna(' ')

In [564]:
len(dadosTreinoBin)

427

In [565]:
len(dadosTesteBin)

200

In [568]:
tweetsParaTreinoBin = dadosTreinoBin['full_text'].values
classesParaTreinoBin = dadosTreinoBin['SentimentoFinal'].values
tweetsParaTesteBin = dadosTesteBin['full_text'].values
classesParaTesteBin = dadosTesteBin['SentimentoFinal'].values


In [569]:
# Instancia o objeto que faz a vetorização dos dados de texto:
tweet_tokenizer = TweetTokenizer() 
vectorizer = CountVectorizer(analyzer="word", tokenizer=tweet_tokenizer.tokenize)

In [570]:
vect_tweetsTreinoBin = vectorizer.fit_transform(tweetsParaTreinoBin) 

In [573]:
classificadorBin = MultinomialNB()
classificadorBin.fit(vect_tweetsTreinoBin, classesParaTreinoBin)

classificadorBinBerno = BernoulliNB()
classificadorBinBerno.fit(vect_tweetsTreinoBin, classesParaTreinoBin) 

classificadorBinComplement = ComplementNB()
classificadorBinComplement.fit(vect_tweetsTreinoBin, classesParaTreinoBin) 

classificadorSVMBin = svm.SVC(kernel='linear')
classificadorSVMBin.fit(vect_tweetsTreinoBin, classesParaTreinoBin)

classificadorLRBin = LogisticRegression(random_state=0).fit(vect_tweetsTreinoBin, classesParaTreinoBin)

In [574]:
vect_tweetsTesteBin = vectorizer.transform(tweetsParaTesteBin) 

In [575]:
resultBin = classificadorBin.predict(vect_tweetsTesteBin)

resultBinBerno = classificadorBinBerno.predict(vect_tweetsTesteBin)

resultBinComplement = classificadorBinComplement.predict(vect_tweetsTesteBin)

resultBinSVM = classificadorSVMBin.predict(vect_tweetsTesteBin)

resultBinLR = classificadorLRBin.predict(vect_tweetsTesteBin)

In [576]:
acc = accuracy_score(classesParaTesteBin, resultBin) * 100
print("{}{} ".format(acc.round(2), "%"))

acc2 = accuracy_score(classesParaTesteBin, resultBinSVM) * 100
print("{}{} ".format(acc2.round(2), "%"))

acc3 = accuracy_score(classesParaTesteBin, resultBinLR) * 100
print("{}{} ".format(acc3.round(2), "%"))

acc4 = accuracy_score(classesParaTesteBin, resultBinBerno) * 100
print("{}{} ".format(acc2.round(2), "%"))

acc5 = accuracy_score(classesParaTesteBin, resultBinComplement) * 100
print("{}{} ".format(acc3.round(2), "%"))

74.5% 
78.0% 
80.0% 
78.0% 
80.0% 


In [577]:
print (pd.crosstab(classesParaTesteBin, resultBin, rownames=['Real'], colnames=['Predito'], margins=True))

#print (pd.crosstab(classesParaTesteBin, resultBinSVM, rownames=['Real'], colnames=['Predito'], margins=True))

#print (pd.crosstab(classesParaTesteBin, resultBinLR, rownames=['Real'], colnames=['Predito'], margins=True))

print (pd.crosstab(classesParaTesteBin, resultBinBerno, rownames=['Real'], colnames=['Predito'], margins=True))

print (pd.crosstab(classesParaTesteBin, resultBinComplement, rownames=['Real'], colnames=['Predito'], margins=True))


Predito   1    2  All
Real                 
1        38   21   59
2        30  111  141
All      68  132  200
Predito   1    2  All
Real                 
1        10   49   59
2         5  136  141
All      15  185  200
Predito   1    2  All
Real                 
1        42   17   59
2        43   98  141
All      85  115  200


In [578]:
print(classification_report(classesParaTesteBin, resultBin, labels = [0,1]))

#print(classification_report(classesParaTesteBin, resultBinSVM, labels = [0,1]))

#print(classification_report(classesParaTesteBin, resultBinLR, labels = [0,1]))

print(classification_report(classesParaTesteBin, resultBinBerno, labels = [0,1]))

print(classification_report(classesParaTesteBin, resultBinComplement, labels = [0,1]))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.56      0.64      0.60        59

   micro avg       0.56      0.64      0.60        59
   macro avg       0.28      0.32      0.30        59
weighted avg       0.56      0.64      0.60        59

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.67      0.17      0.27        59

   micro avg       0.67      0.17      0.27        59
   macro avg       0.33      0.08      0.14        59
weighted avg       0.67      0.17      0.27        59

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.49      0.71      0.58        59

   micro avg       0.49      0.71      0.58        59
   macro avg       0.25      0.36      0.29        59
weighted avg       0.49      0.71      0.58        59



/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(a

#### Agora faremos o treinamento com o dataset de assuntos diversos sem os tweets neutros.

In [582]:
dadosTreinoDiversosBin = pd.read_excel('TabelaDiversosBinPreProcessados.xlsx', engine='openpyxl').fillna(' ')

In [583]:
tweetsParaTreinoDiversosBin = dadosTreinoDiversosBin['full_text'].values
classesParaTreinoDiversosBin = dadosTreinoDiversosBin['SentimentoFinal'].values

In [584]:
qtdTweetsTreinoBin = len(dadosTreinoDiversosBin)
qtdTweetsTreinoBin

556

In [585]:
vect_tweetsTreinoDiversosBin = vectorizer.fit_transform(tweetsParaTreinoDiversosBin)  #treina com diversos

In [586]:
classificadorDiversosBin = MultinomialNB()
classificadorDiversosBin.fit(vect_tweetsTreinoDiversosBin, classesParaTreinoDiversosBin)

classificadorDiversosBinBerno = BernoulliNB()
classificadorDiversosBinBerno.fit(vect_tweetsTreinoDiversosBin, classesParaTreinoDiversosBin) 

classificadorDiversosBinComplement = ComplementNB()
classificadorDiversosBinComplement.fit(vect_tweetsTreinoDiversosBin, classesParaTreinoDiversosBin) 

classificadorDiversosSVMBin = svm.SVC(kernel='linear')
classificadorDiversosSVMBin.fit(vect_tweetsTreinoDiversosBin, classesParaTreinoDiversosBin)

classificadorDiversosLRBin = LogisticRegression(random_state=0).fit(vect_tweetsTreinoDiversosBin, classesParaTreinoDiversosBin)

In [587]:
vect_tweetsTesteBin = vectorizer.transform(tweetsParaTesteBin) 

In [590]:
resultDiversosBin = classificadorDiversosBin.predict(vect_tweetsTesteBin)

resultDiversosBinBerno = classificadorDiversosBinBerno.predict(vect_tweetsTesteBin)

resultDiversosBinComplement = classificadorDiversosBinComplement.predict(vect_tweetsTesteBin)

resultDiversosBinSVM = classificadorDiversosSVMBin.predict(vect_tweetsTesteBin)

resultDiversosBinLR = classificadorDiversosLRBin.predict(vect_tweetsTesteBin)

In [591]:
acc = accuracy_score(classesParaTesteBin, resultDiversosBin) * 100
print("{}{} ".format(acc.round(2), "%"))

acc2 = accuracy_score(classesParaTesteBin, resultDiversosBinSVM) * 100
print("{}{} ".format(acc2.round(2), "%"))

acc3 = accuracy_score(classesParaTesteBin, resultDiversosBinLR) * 100
print("{}{} ".format(acc3.round(2), "%"))

acc4 = accuracy_score(classesParaTesteBin, resultDiversosBinBerno) * 100
print("{}{} ".format(acc2.round(2), "%"))

acc5 = accuracy_score(classesParaTesteBin, resultDiversosBinComplement) * 100
print("{}{} ".format(acc3.round(2), "%"))

69.0% 
80.0% 
78.5% 
80.0% 
78.5% 


In [592]:
print (pd.crosstab(classesParaTesteBin, resultDiversosBin, rownames=['Real'], colnames=['Predito'], margins=True))

#print (pd.crosstab(classesParaTesteBin, resultDiversosBinSVM, rownames=['Real'], colnames=['Predito'], margins=True))

#print (pd.crosstab(classesParaTesteBin, resultDiversosBinLR, rownames=['Real'], colnames=['Predito'], margins=True))

print (pd.crosstab(classesParaTesteBin, resultDiversosBinBerno, rownames=['Real'], colnames=['Predito'], margins=True))

print (pd.crosstab(classesParaTesteBin, resultDiversosBinComplement, rownames=['Real'], colnames=['Predito'], margins=True))


Predito   1    2  All
Real                 
1        36   23   59
2        39  102  141
All      75  125  200
Predito   1    2  All
Real                 
1         7   52   59
2         5  136  141
All      12  188  200
Predito    1   2  All
Real                 
1         45  14   59
2         62  79  141
All      107  93  200


In [593]:
print(classification_report(classesParaTesteBin, resultDiversosBin, labels = [0,1]))

#print(classification_report(classesParaTesteBin, resultDiversosBinSVM, labels = [0,1]))

#print(classification_report(classesParaTesteBin, resultDiversosBinLR, labels = [0,1]))

print(classification_report(classesParaTesteBin, resultDiversosBinBerno, labels = [0,1]))

print(classification_report(classesParaTesteBin, resultDiversosBinComplement, labels = [0,1]))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.48      0.61      0.54        59

   micro avg       0.48      0.61      0.54        59
   macro avg       0.24      0.31      0.27        59
weighted avg       0.48      0.61      0.54        59

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.58      0.12      0.20        59

   micro avg       0.58      0.12      0.20        59
   macro avg       0.29      0.06      0.10        59
weighted avg       0.58      0.12      0.20        59

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.42      0.76      0.54        59

   micro avg       0.42      0.76      0.54        59
   macro avg       0.21      0.38      0.27        59
weighted avg       0.42      0.76      0.54        59



/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(a

## Teste cruzado com  #netflix binário

In [678]:
dadosTreinoDiversosBin = pd.read_excel('/Users/gustavoduarte/Desktop/TCC-analise-sentimentos-main/TabelaDiversosBinPreProcessados.xlsx',  engine='openpyxl').fillna(' ')
dadosTesteNetflixBin = pd.read_excel('/Users/gustavoduarte/Desktop/TCC-analise-sentimentos-main/TabelaNetflixBinPreProcessados.xlsx',  engine='openpyxl').fillna(' ')

In [679]:
tweetsParaTreinoDivBin = dadosTreinoDiversosBin['full_text'].values
classesParaTreinoDivBin = dadosTreinoDiversosBin['SentimentoFinal'].values
tweetsParaTesteNetflixBin = dadosTesteNetflixBin['full_text'].values
classesParaTesteNetflixBin = dadosTesteNetflixBin['SentimentoFinal'].values

In [680]:
qtdTweetsTreinoBin = len(dadosTreinoDiversosBin)
qtdTweetsTreinoBin

556

In [681]:
qtdTweetsTesteBin = len(dadosTesteNetflixBin)
qtdTweetsTesteBin

239

In [682]:
# Instancia o objeto que faz a vetorização dos dados de texto:
tweet_tokenizer_bin = TweetTokenizer() 
vectorizerBin = CountVectorizer(analyzer="word", tokenizer=tweet_tokenizer_bin.tokenize)

In [683]:
vect_tweetsTreinoDivBin = vectorizerBin.fit_transform(tweetsParaTreinoDivBin)


In [684]:
classificadorValidBin = MultinomialNB()
classificadorValidBin.fit(vect_tweetsTreinoDivBin, classesParaTreinoDivBin)

classificadorValidBernoBin = BernoulliNB()
classificadorValidBernoBin.fit(vect_tweetsTreinoDivBin, classesParaTreinoDivBin) 

classificadorValidComplementBin = ComplementNB()
classificadorValidComplementBin.fit(vect_tweetsTreinoDivBin, classesParaTreinoDivBin) 

classificadorValidSVMBin = svm.SVC(kernel='linear')
classificadorValidSVMBin.fit(vect_tweetsTreinoDivBin, classesParaTreinoDivBin)

classificadorValidLRBin = LogisticRegression(random_state=0).fit(vect_tweetsTreinoDivBin, classesParaTreinoDivBin)

In [685]:
vect_tweetsTesteNetflixBin = vectorizerBin.transform(tweetsParaTesteNetflixBin) 

In [686]:
resultValidBin = classificadorValidBin.predict(vect_tweetsTesteNetflixBin)

resultValidBernoBin = classificadorValidBernoBin.predict(vect_tweetsTesteNetflixBin)

resultValidComplementBin = classificadorValidComplementBin.predict(vect_tweetsTesteNetflixBin)

resultValidSVMBin = classificadorValidSVMBin.predict(vect_tweetsTesteNetflixBin)

resultValidLRBin = classificadorValidLRBin.predict(vect_tweetsTesteNetflixBin)

In [687]:
acc = accuracy_score(classesParaTesteNetflixBin, resultValidBin) * 100
print("{}{} ".format(acc.round(2), "%"))

acc2 = accuracy_score(classesParaTesteNetflixBin, resultValidSVMBin) * 100
print("{}{} ".format(acc2.round(2), "%"))

acc3 = accuracy_score(classesParaTesteNetflixBin, resultValidLRBin) * 100
print("{}{} ".format(acc3.round(2), "%"))

acc4 = accuracy_score(classesParaTesteNetflixBin, resultValidBernoBin) * 100
print("{}{} ".format(acc.round(2), "%"))

acc5 = accuracy_score(classesParaTesteNetflixBin, resultValidComplementBin) * 100
print("{}{} ".format(acc.round(2), "%"))

66.95% 
67.36% 
70.71% 
66.95% 
66.95% 


In [688]:
print (pd.crosstab(classesParaTesteNetflixBin, resultValidBin, rownames=['Real'], colnames=['Predito'], margins=True))

print (pd.crosstab(classesParaTesteNetflixBin, resultValidBernoBin, rownames=['Real'], colnames=['Predito'], margins=True))

print (pd.crosstab(classesParaTesteNetflixBin, resultValidComplementBin, rownames=['Real'], colnames=['Predito'], margins=True))


#print (pd.crosstab(classesParaTesteNetflixBin, resultValidSVMBin, rownames=['Real'], colnames=['Predito'], margins=True))

#print (pd.crosstab(classesParaTesteNetflixBin, resultValidLRBin, rownames=['Real'], colnames=['Predito'], margins=True))

Predito   1    2  All
Real                 
1        13   53   66
2        26  147  173
All      39  200  239
Predito  1    2  All
Real                
1        2   64   66
2        1  172  173
All      3  236  239
Predito   1    2  All
Real                 
1        23   43   66
2        50  123  173
All      73  166  239


## Por fim, vamos validar os resultados realizando o treinamento com os tweets de assuntos diversos e o teste com os tweets #netflix

Esta etapa de treinamento/teste é como as anteriores, logo, não há necessidade de explicar cada passo.

In [614]:
dadosTreinoDiversos = pd.read_excel('/Users/gustavoduarte/Desktop/TCC-analise-sentimentos-main/TabelaDiversosPreProcessados.xlsx',  engine='openpyxl').fillna(' ')
dadosTesteNetflix = pd.read_excel('/Users/gustavoduarte/Desktop/TCC-analise-sentimentos-main/TabelaNetflixPreProcessados.xlsx',  engine='openpyxl').fillna(' ')

In [615]:
tweetsParaTreinoDiv = dadosTreinoDiversos['full_text'].values
classesParaTreinoDiv = dadosTreinoDiversos['SentimentoFinal'].values
tweetsParaTesteNetflix = dadosTesteNetflix['full_text'].values
classesParaTesteNetflix = dadosTesteNetflix['SentimentoFinal'].values

In [616]:
qtdTweetsTreino = len(dadosTreinoDiversos)
qtdTweetsTreino

700

In [617]:
qtdTweetsTeste = len(dadosTesteNetflix)
qtdTweetsTeste

300

In [618]:
# Instancia o objeto que faz a vetorização dos dados de texto:
tweet_tokenizer = TweetTokenizer() 
vectorizer = CountVectorizer(analyzer="word", tokenizer=tweet_tokenizer.tokenize)

In [619]:
vect_tweetsTreinoDiv = vectorizer.fit_transform(tweetsParaTreinoDiv)



In [620]:
classificadorValid = MultinomialNB()
classificadorValid.fit(vect_tweetsTreinoDiv, classesParaTreinoDiv)

classificadorValidBerno = BernoulliNB()
classificadorValidBerno.fit(vect_tweetsTreinoDiv, classesParaTreinoDiv) 

classificadorValidComplement = ComplementNB()
classificadorValidComplement.fit(vect_tweetsTreinoDiv, classesParaTreinoDiv) 

classificadorValidSVM = svm.SVC(kernel='linear')
classificadorValidSVM.fit(vect_tweetsTreinoDiv, classesParaTreinoDiv)

classificadorValidLR = LogisticRegression(random_state=0).fit(vect_tweetsTreinoDiv, classesParaTreinoDiv)

In [621]:
vect_tweetsTesteNetflix = vectorizer.transform(tweetsParaTesteNetflix) 

In [622]:
resultValid = classificadorValid.predict(vect_tweetsTesteNetflix)

resultValidBerno = classificadorValidBerno.predict(vect_tweetsTesteNetflix)

resultValidComplement = classificadorValidComplement.predict(vect_tweetsTesteNetflix)

resultValidSVM = classificadorValidSVM.predict(vect_tweetsTesteNetflix)

resultValidLR = classificadorValidLR.predict(vect_tweetsTesteNetflix)

In [623]:
acc = accuracy_score(classesParaTesteNetflix, resultValid) * 100
print("{}{} ".format(acc.round(2), "%"))

acc2 = accuracy_score(classesParaTesteNetflix, resultValidSVM) * 100
print("{}{} ".format(acc2.round(2), "%"))

acc3 = accuracy_score(classesParaTesteNetflix, resultValidLR) * 100
print("{}{} ".format(acc3.round(2), "%"))

acc4 = accuracy_score(classesParaTesteNetflix, resultValidBerno) * 100
print("{}{} ".format(acc.round(2), "%"))

acc5 = accuracy_score(classesParaTesteNetflix, resultValidComplement) * 100
print("{}{} ".format(acc.round(2), "%"))

54.0% 
56.33% 
57.33% 
54.0% 
54.0% 


In [624]:
print (pd.crosstab(classesParaTesteNetflix, resultValid, rownames=['Real'], colnames=['Predito'], margins=True))

print (pd.crosstab(classesParaTesteNetflix, resultValidBerno, rownames=['Real'], colnames=['Predito'], margins=True))

print (pd.crosstab(classesParaTesteNetflix, resultValidComplement, rownames=['Real'], colnames=['Predito'], margins=True))


#print (pd.crosstab(classesParaTesteNetflix, resultValidSVM, rownames=['Real'], colnames=['Predito'], margins=True))

#print (pd.crosstab(classesParaTesteNetflix, resultValidLR, rownames=['Real'], colnames=['Predito'], margins=True))

Predito   0   1    2  All
Real                     
0        15   5   41   61
1         6   9   51   66
2        10  25  138  173
All      31  39  230  300
Predito  1    2  All
Real                
0        0   61   61
1        2   64   66
2        0  173  173
All      2  298  300
Predito   0   1    2  All
Real                     
0        25   8   28   61
1        20  17   29   66
2        32  37  104  173
All      77  62  161  300


In [559]:
print(classification_report(classesParaTesteNetflix, resultValid, labels = [0,1,2]))
print(classification_report(classesParaTesteNetflix, resultValidBerno, labels = [0,1,2]))
print(classification_report(classesParaTesteNetflix, resultValidComplement, labels = [0,1,2]))


#print(classification_report(classesParaTesteNetflix, resultValidSVM, labels = [0,1,2]))

#print(classification_report(classesParaTesteNetflix, resultValidLR, labels = [0,1,2]))

              precision    recall  f1-score   support

           0       0.48      0.25      0.33        61
           1       0.23      0.14      0.17        66
           2       0.60      0.80      0.68       173

    accuracy                           0.54       300
   macro avg       0.44      0.39      0.39       300
weighted avg       0.50      0.54      0.50       300

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        61
           1       1.00      0.03      0.06        66
           2       0.58      1.00      0.73       173

    accuracy                           0.58       300
   macro avg       0.53      0.34      0.26       300
weighted avg       0.55      0.58      0.44       300

              precision    recall  f1-score   support

           0       0.32      0.41      0.36        61
           1       0.27      0.26      0.27        66
           2       0.65      0.60      0.62       173

    accuracy        

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

Abaixo, o treinamento/teste aonde sem tweets neutros.

In [89]:
dadosTreinoDiversosBin = pd.read_excel('TabelaDiversosPreProcessadosBin.xlsx').fillna(' ')
dadosTesteNetflixBin = pd.read_excel('TabelaNetflixPreProcessadosBin.xlsx').fillna(' ')

In [90]:
tweetsParaTreinoDiversosBin = dadosTreinoDiversosBin['full_text'].values
classesParaTreinoDiversosBin = dadosTreinoDiversosBin['SentimentoFinal'].values
tweetsParaTesteNetflixBin = dadosTesteNetflixBin['full_text'].values
classesParaTesteNetflixBin = dadosTesteNetflixBin['SentimentoFinal'].values

In [91]:
# Instancia o objeto que faz a vetorização dos dados de texto:
tweet_tokenizer = TweetTokenizer() 
vectorizer = CountVectorizer(analyzer="word", tokenizer=tweet_tokenizer.tokenize)

In [92]:
vect_tweetsTreinoBin = vectorizer.fit_transform(tweetsParaTreinoDiversosBin) 

In [93]:
classificadorBinValid = MultinomialNB()
classificadorBinValid.fit(vect_tweetsTreinoBin, classesParaTreinoDiversosBin)

classificadorBinValidSVM = svm.SVC(kernel='linear')
classificadorBinValidSVM.fit(vect_tweetsTreinoBin, classesParaTreinoDiversosBin)

classificadorBinValidLR = LogisticRegression(random_state=0).fit(vect_tweetsTreinoBin, classesParaTreinoDiversosBin)

In [94]:
vect_tweetsTesteNetflixBin = vectorizer.transform(tweetsParaTesteNetflixBin) 

In [99]:
resultValidBin = classificadorBinValid.predict(vect_tweetsTesteNetflixBin)

resultValidBinSVM = classificadorBinValidSVM.predict(vect_tweetsTesteNetflixBin)

resultValidBinLR = classificadorBinValidLR.predict(vect_tweetsTesteNetflixBin)

In [100]:
acc = accuracy_score(classesParaTesteNetflixBin, resultValidBin) * 100
print("{}{} ".format(acc.round(2), "%"))

acc2 = accuracy_score(classesParaTesteNetflixBin, resultValidBinSVM) * 100
print("{}{} ".format(acc2.round(2), "%"))

acc3 = accuracy_score(classesParaTesteNetflixBin, resultValidBinLR) * 100
print("{}{} ".format(acc3.round(2), "%"))

68.5% 
65.35% 
66.14% 


In [101]:
print (pd.crosstab(classesParaTesteNetflixBin, resultValidBin, rownames=['Real'], colnames=['Predito'], margins=True))

print (pd.crosstab(classesParaTesteNetflixBin, resultValidBinSVM, rownames=['Real'], colnames=['Predito'], margins=True))

print (pd.crosstab(classesParaTesteNetflixBin, resultValidBinLR, rownames=['Real'], colnames=['Predito'], margins=True))

Predito   0   1  All
Real                
0        44  18   62
1        22  43   65
All      66  61  127
Predito   0   1  All
Real                
0        35  27   62
1        17  48   65
All      52  75  127
Predito   0   1  All
Real                
0        33  29   62
1        14  51   65
All      47  80  127


In [102]:
print(classification_report(classesParaTesteNetflixBin, resultValidBin, labels = [0,1]))

print(classification_report(classesParaTesteNetflixBin, resultValidBinSVM, labels = [0,1]))

print(classification_report(classesParaTesteNetflixBin, resultValidBinLR, labels = [0,1]))

             precision    recall  f1-score   support

          0       0.67      0.71      0.69        62
          1       0.70      0.66      0.68        65

avg / total       0.69      0.69      0.68       127

             precision    recall  f1-score   support

          0       0.67      0.56      0.61        62
          1       0.64      0.74      0.69        65

avg / total       0.66      0.65      0.65       127

             precision    recall  f1-score   support

          0       0.70      0.53      0.61        62
          1       0.64      0.78      0.70        65

avg / total       0.67      0.66      0.66       127




###### Obrigado por olhar o meu notebook! Fique livre para copiá-lo, altera-lo e utiliza-lo em seus estudos/projetos 